In this assignment you will implement one or more algorithms for the all-pairs shortest-path problem. Here are data files describing three graphs:
- g1.txt
- g2.txt
- g3.txt

The first line indicates the number of vertices and edges, respectively. Each subsequent line describes an edge (the first two numbers are its tail and head, respectively) and its length (the third number). NOTE: some of the edge lengths are negative. NOTE: These graphs may or may not have negative-cost cycles.

Your task is to compute the "shortest shortest path". Precisely, you must first identify which, if any, of the three graphs have no negative cycles. For each such graph, you should compute all-pairs shortest paths and remember the smallest one [i.e., compute $\min_{u,v \in V} d(u,v)$, where $d(u,v)$ denotes the shortest-path distance from $u$ to $v$].

If each of the three graphs has a negative-cost cycle, then enter "NULL" in the box below. If exactly one graph has no negative-cost cycles, then enter the length of its shortest shortest path in the box below. If two or more of the graphs have no negative-cost cycles, then enter the smallest of the lengths of their shortest shortest paths in the box below.

OPTIONAL: You can use whatever algorithm you like to solve this question. If you have extra time, try comparing the performance of different all-pairs shortest-path algorithms!

OPTIONAL: Here is a bigger data set to play with.
- large.txt

For fun, try computing the shortest shortest path of the graph in the file above.

# All-Pairs Shortest Paths

## Input
Directed graph $G = (V, E)$ with edge costs $c_e$ for each edge $e \in E$.

## Output
Either (1) compute the length of a shortest $u \rightarrow v$ path for all pairs of vertices $u, v \in V$,
or (2) correctly report that $G$ contains a negative cycle.

# Floyd-Warshall Algorithm

## Optimal Substructure
Order the vertices $V = \{1, 2, \dots, n\}$ arbitrarily. Let $V^{(k)} = \{1,2,\dots,k\}$.
Suppose G has no negative cycle.
Given a source vertex $u$, a destination vertex $v$, and $k \in \{1,2,\dots,n\}$, let $P$ be the shortest cycle-free $u \rightarrow v$ path with __*all internal nodes*__ in $V^{(k)}$, then
1. if $k {\not \in} P$, $P$ is a shortest cycle-free $u \rightarrow v$ path with all internal vertices in $V^{(k-1)}$.

2. if $k \in P$, then $P$ can be separated to two parts:
    - $P_1$: the shortest cycle-free $u \rightarrow k$ path with all internal nodes in $V^{(k-1)}$.
    - $P_2$: the shortest cycle-free $k \rightarrow v$ path with all internal nodes in $V^{(k-1)}$.

## Algorithm
```python
A = 3D array indexed by k, u, v
for u in V:
    for v in V:
        if u == v:
            A[u, v, 0] = 0
        elif (u, v) in E:
            A[u, v, 0] = c[u, v]
        else:
            A[u, v, 0] = float("inf")

for k in range(1, n + 1):
    for u in range(1, n + 1):
        for v in range(1, n + 1):
            # case 1: inherit from A[u, v, k - 1]
            # case 2: separated paths
            A[u, v, k] = min(A[u, v, k - 1], A[u, k, k - 1] + A[k, v, k - 1])

negative_cycle = False
for i in range(1, n + 1):
    if A[i, i, n] < 0:
        negative_cycle = True
        break
```
The whole path can be reconstructed by saving another matrix `B` with `B[u][v] = k` whenever case 2 is invoked.

In [1]:
def readfile(filename):
    nv, ne = 0, 0
    edges = {}
    for n, line in enumerate(open(filename)):
        line = map(int, line.split())
        if n == 0:
            nv, ne = line
        else:
            u, v, ce = line
            try:
                if ce > edges[(u, v)]:
                    edges[(u, v)] = ce
            except:
                edges[(u, v)] = ce
    
    return nv, ne, edges

import numpy as np
def floyd_warshall(nv, graph):
    """ Floyd-Warshall on APSP.
    nv -- number of vertices
    graph -- graph represented by {(tail, head): dist} map
    
    Return the value of all-pairs shortest path.
    """
    
    A = np.full((nv, nv), np.inf, dtype=float)
    A[np.diag_indices(nv)] = 0
    for e, ce in graph.iteritems():
        u, v = e
        A[u - 1][v - 1] = ce
    
    for k in range(nv):
        shifted = A[k, :][None, :] + A[:, k][:, None] # a direct sum of k-th row and k-th column
        changed = shifted < A
        if changed.any():
            A = np.where(changed, shifted, A)
    
    negative_cycle = (A.diagonal() < 0).any()
    vmin = float("inf")
    
    if negative_cycle:
        print "There is at least one negative cycle."
    else:
        vmin = np.min(A)

    return vmin

In [2]:
# timer grabbed from 
# https://stackoverflow.com/questions/7370801/measure-time-elapsed-in-python
from timeit import default_timer as timer
class benchmark(object):
    def __init__(self, msg, fmt="%0.3g"):
        self.msg = msg
        self.fmt = fmt

    def __enter__(self):
        self.start = timer()
        return self

    def __exit__(self, *args):
        t = timer() - self.start
        print(("%s: " + self.fmt + " seconds") % (self.msg, t))
        self.time = t

In [3]:
# test case -- -242
nv, ne, G = readfile("test.txt")
with benchmark("Floyd-Warshall on test.txt") as r:
    vmin = floyd_warshall(nv, G)
assert vmin == -242, "Floyd-Warshall does not pass the test case!"
print "Floyd-Warshall passes the test case!"

Floyd-Warshall on test.txt: 0.00106 seconds
Floyd-Warshall passes the test case!


In [4]:
files = ["g1.txt", "g2.txt", "g3.txt"]
files = ["g3.txt"]
for filename in files:
    nv, ne, G = readfile(filename)
    with benchmark("Floyd-Warshall on {0}".format(filename)) as r:
        vmin = floyd_warshall(nv, G)
#         print "Shortest path value of {0} is {1}.".format(filename, vmin)

Floyd-Warshall on g3.txt: 7.87 seconds


In [5]:
# n = 20000, m = 999387
# Requires ~3 GB to store A of size n by n. Not very practicle on my computer.

# with benchmark("Read file large.txt") as r:
#     nv, ne, G = readfile("large.txt")
# with benchmark("Floyd-Warshall on large.txt") as r:
#     vmin = floyd_warshall(nv, G)
# print "Shortest path value of {0} is {1}.".format("large.txt", vmin)


##########################
# Results on our cluster #
##########################

# Read file large.txt: 2.79 seconds                                                                         
# Floyd-Warshall on large.txt: 4.7e+04 seconds
# Shortest path value of large.txt is -6.0.

# Johnson's Algorithm on APSP

## Key Idea: Reweighting
Each vertex $v$ is associated with a weight $p_v$.
We can add $p_u - p_v$ to **every** $u \rightarrow v$ path without changing the shortest path.
The vertex weight $p_v$ is chosen as the shortest length of $s \rightarrow v$ path, where $s$ is a fictitious node with zero edge cost to all vertices in $G$: $c_{s - v} = 0, \forall v \in V$.
Note that adding $s$ does not add any new $u \rightarrow v$ paths for any $u, v \in G$.

## Algorithm
Given a directed graph $G = (V, E)$ with general edge lengths $c_e$.
1. Form $G'$ by adding a new vertex $s$ and a new edge $(s, v)$ with length 0 for all $v \in G$.

2. Run Bellman-Ford on $G'$ with source vertex $s$.
   Report any negative-cost cycle if detected because it must lie in $G$.
   
3. For each $v \in G$, define $p_v = $ the length of shortest $s \rightarrow v$ path in $G'$.
   Define $c_{u-v}' = c_{u-v} + p_u - p_v$ for each edge $(u, v) \in G$.
   
4. For each vertex $u$ of $G$, compute the shortest path distance $d'[u, v]$ for all $v \in G$ with souce vertex $u$ and edge lengths $\{c_e'\}$ via Dijkstra's algorithm.

5. For each pair $u, v \in G$, return the shortest-path distance $d[u,v] = d'[u,v] - p_u + p_v$.

# Bellman-Ford Algorithm

## Optimal Substructure
For every $v \in V, i \in \{1,2,\dots\}$, let $P$ be the shortest $s \rightarrow v$ path with __*at most $i$ edges*__. (Cycles are permitted.)
1. If $P$ has $\leq (i - 1)$ edges, it is a shortest $s \rightarrow v$ path with $\leq (i - 1)$ edges.
2. If $P$ has $i$ edges with last hop $(w,v)$, then $P'$ is a shortest $s \rightarrow w$ path with $\leq (i - 1)$ edges.


## Recurrence

Let $L_{i,v}$ be the minimum length of a $s \rightarrow v$ path with $\leq i$ edges.
Cycles are allowed and $L_{i,v} = +\infty$ if there is no $s \rightarrow v$ path with $\leq i$ edges.
For every $v \in V, i \in {1,2,\dots}$,
\begin{align}
  L_{i,v} = \min[ L_{(i-1), v}, \min_{(w,v) \in E} ( L_{(i-1), w} + c_{wv} ) ].
\end{align}
The above equation suggests a brute-force search from the only [1 + in-degree($v$)] candidates.

If there is no negative cycles in $G$, we only need to solve subproblems up to $i = n - 1$.
The negative cycles can be found by running one more iteration because
\begin{align}
  \text{$G$ has no negative-cost cycle that is reachable from $s$}\, \Longleftrightarrow \, A[n − 1, v] = A[n, v],  \forall v \in V.
\end{align}


## Algorithm

```python
A = 2-D array indexed by i and v

for v in range(n):
    if v == s:
        A[0, s] = 0
    else:
        A[0, v] = float("inf")

for i in range(1, n):
    for v in range(n):
        changed = float("inf")
        for (w, v) in E:
            c_ = A[i - 1, w] + c[w, v]
            if c_ < changed:
                changed = c_
        A[i, v] = min(A[i - 1, v], changed)

# last iteration to detect negative cycles
for v in range(n):
    changed = float("inf")
    for (w, v) in E:
        c_ = A[n - 1, w] + c[w, v]
        if c_ < changed:
            changed = c_
    if changed < A[n - 1, v]:
        print "Negative cycles detected."

return A[n - 1]
```


## Optimizations

### Stopping early
Suppose for some $j < n - 1$, $A[j, v] = A[j - 1, v]$ for all vertices $v$.
Then all future $A[j, v]$ will be the same for all $v$.

### Space optimizations

Only need the $A[i − 1, v]$ to compute the $A[i, v]$.

To compute predecessors, we compute a second table $B$, where $B[i, v]$ is the 2nd-to-last vertex on the shortest $s \rightarrow v$ path with $\leq i$ edges (or NULL if no such paths exist), i.e., update $B[i, v]$ to the minimum $w$ whenever case 2 occurs.

Assuming the input graph $G$ has no negative cycles, then the whole $s \rightarrow v$ path is constructed by tracing back predecessor pointers -- the $B[n - 1, v]$ (i.e., last hop of a shortest $s \rightarrow v$ path) -- from $v$ to $s$.

To reconstruct a negative-cost cycle, use depth-first search to check for a cycle of predecessor pointers after each round (must be a negative cost cycle).

In [6]:
# This block implements the Bellman-Ford algorithm.

def bellman_ford(s, nv, graph):
    """ Bellman-Ford on shortest path problem.
    s -- source vertex
    nv -- number of vertices
    graph -- graph represented by {(tail, head): dist} map
    
    Return the value of all-pairs shortest path.
    """
    A = np.full((nv), np.inf, dtype=float)
    A[s - 1] = 0
    
    in_degree = {}
    for tail, head in graph.keys():
        try:
            in_degree[head - 1].append(tail - 1) # shift to zero-based index to simplify my life
        except:
            in_degree[head - 1] = [tail - 1]
    
    # detect negative cycle when i = nv
    for i in range(1, nv + 1):
        shifted = np.full((nv), np.inf, dtype=float)
        
        # brute-force search
        for v in range(nv):
            try:
                vmin = float("inf")
                for w in in_degree[v]:
                    vw = A[w] + graph[(w + 1, v + 1)]
                    if vw < vmin:
                        vmin = vw
                shifted[v] = vmin
            except:
                pass
        
        changed = shifted < A
        if changed.any():
            A = np.where(changed, shifted, A)
        else:
            # stopping early
            return A
    
    # negative cycle detected
    print "There is at least one negative cycle."
    return float("inf")

# some tests
with benchmark("Bellman-Ford on test.txt") as r:
    nv, ne, G = readfile("test.txt")
    A = bellman_ford(2, nv, G)
    print A
# [  inf    0.   inf   inf   73.   inf   90.   inf   inf   inf   -9.  -31.
#    66.   13.    3.  -72.  -53.  -46. -137.  -56.   91.   36.   19.   inf
#     6.  -48.  -87. -104. -193. -145.   75.   17.]

print
with benchmark("Bellman-Ford on test1.txt") as r:
    nv, ne, G = readfile("test1.txt")
    A = bellman_ford(2, nv, G)
    print A

[  inf    0.   inf   inf   73.   inf   90.   inf   inf   inf   -9.  -31.
   66.   13.    3.  -72.  -53.  -46. -137.  -56.   91.   36.   19.   inf
    6.  -48.  -87. -104. -193. -145.   75.   17.]
Bellman-Ford on test.txt: 0.0115 seconds

There is at least one negative cycle.
inf
Bellman-Ford on test1.txt: 0.211 seconds


In [7]:
# This block implements the Dijkstra algorithm.

# I am not sure how to keep track of indices change when using heapq.
# Go back to my implementation of heap.

class dijkstra_heap:
    """ Binary heap for Dijkstra's algorithm.
    array -- a list of node-distance pair
    """
    
    def __init__(self, array=[]):
        self.data = []         # node-distance pairs
        self.indices = {}      # map node to its index
        self.size = len(array) # the number of nodes currently in the heap
        
        if self.size != 0:
            for i, pair in enumerate(array):
                node, distance = pair
                self.data.append(pair)
                self.indices[node] = i
            self.heapify()
        
        return
    
    def __repr__(self):
        return "Key value:\n" + str(self.data) + "\nNode locataion:\n" + str(self.indices)
    
    def __contains__(self, node):
        return node in self.indices
    
    def is_empty(self):
        return self.size == 0
    
    def validate_index(self, i):
        """ Check if index i lies in bound. """
        if i < 0 or i >= self.size:
            print "Index i = {0}, size of heap: {1}".format(i, self.size)
            raise ValueError("Index out of range.")
        return
    
    def parent_index(self, i):
        """ Return the parent index of child i. """
        self.validate_index(i)
        return (i - 1) / 2 if i != 0 else 0
    
    def children_indices(self, i):
        """ Return the children indices of parent i. """
        self.validate_index(i)
        return 2 * i + 1, 2 * i + 2
    
    def is_leaf(self, i):
        """ Check if index i is a leaf or not. """
        self.validate_index(i)
        c1, c2 = self.children_indices(i)
        return c1 >= self.size and c2 >= self.size
    
    def one_child(self, i):
        """ Check if parent of index i has only one child. """
        self.validate_index(i)
        c1, c2 = self.children_indices(i)
        return c1 < self.size and c2 >= self.size
    
    def min_value_child(self, i):
        """ Return the child index of parent i with the smaller value. """
        self.validate_index(i)
        c1, c2 = self.children_indices(i)
        
        c = None
        if c2 < self.size: # node i has two children
            c = c1
            if self.data[c1][1] > self.data[c2][1]:
                c = c2
        else:
            if c1 < self.size: # node i has only one child
                c = c1
            else:              # node i is a leaf
                c = i
        return c
    
    def up_heapify(self, i):
        """ Bubble up from index i. """
        self.validate_index(i)
        ic = i
        ip = self.parent_index(ic)
        while self.data[ic][1] < self.data[ip][1]:
            node_c, node_p = self.data[ic][0], self.data[ip][0]
            self.data[ic], self.data[ip] = self.data[ip], self.data[ic] # swap data
            self.indices[node_c], self.indices[node_p] = ip, ic         # update index map
            ic = ip
            ip = self.parent_index(ic)
        return
    
    def down_heapify(self, i):
        """ Bubble down from index i. """
        self.validate_index(i)
        ip = i
        ic = self.min_value_child(ip)
        while self.data[ic][1] < self.data[ip][1]:
            node_c, node_p = self.data[ic][0], self.data[ip][0]
            self.data[ic], self.data[ip] = self.data[ip], self.data[ic] # swap data
            self.indices[node_c], self.indices[node_p] = ip, ic         # update index map
            ip = ic
            ic = self.min_value_child(ip)
        return
    
    def heapify(self):
        """ Heapify the current self. """
        start = self.parent_index(self.size - 1)
        while start >= 0:
            self.down_heapify(start)
            start -= 1
        return
    
    def insert(self, pair):
        self.data.append(pair)
        self.indices[pair[0]] = self.size
        self.size += 1
        self.up_heapify(self.size - 1)
        return
    
    def get_min(self):
        return self.data[0]
    
    def get_value(self, node):
        if node in self.indices:
            i = self.indices[node]
            return self.data[i][1]
        else:
            raise ValueError("Requested node not in the heap!")
    
    def extract_min(self):
        if self.size == 0:
            raise ValueError("Cannot extract min from an empty heap!")
        
        m_node, m_value = self.get_min()
        self.data[0] = self.data[-1]
        self.data.pop()
        self.size -= 1
        self.indices.pop(m_node)
        if self.size > 0:
            self.indices[self.data[0][0]] = 0
            self.down_heapify(0)
        return (m_node, m_value)
    
    def update_node_value(self, node, value):
        if node in self.indices:
            i = self.indices[node]
            self.data[i] = (node, value)
            
            ip = self.parent_index(i)
            ic = self.min_value_child(i)
            if value < self.data[ip][1]:
                self.up_heapify(i)
            elif value > self.data[ic][1]:
                self.down_heapify(i)
        else:
            raise ValueError("Requested node not in the heap!")
        return

def init_infty_dijkstra_heap(s, nv):
    """ Initialization step of Dijkstra algorithm.
    s -- source vertex name (corresponding index in A is s - 1)
    nv -- number of vertices
    """
    
    # final shortest-path for all nodes
    A = np.full((nv), np.inf, dtype=float)
    A[s - 1] = 0
    
    # nodes currently not in A
    q = [(s, 0)]
    
    for node in range(1, nv + 1):
        if node != s:
            q.append((node, A[node - 1]))
    
    # initialize binary heap
    Q = dijkstra_heap(q)
    
    return A, Q

def dijkstra(G, nv, s):
    """ Find the shorest-path using Dijkstra algorithm
    G -- graph represented by {tail: [(head, distance)]}
    s -- source vertex name
    
    Return the shorst-path map of all nodes.
    """
    
    # A: final shortest-path (a numpy array of size nv), Q: binary heap
    A, Q = init_infty_dijkstra_heap(s, nv)

    while not Q.is_empty():
        u, ud = Q.extract_min()
        for v, d in G[u]:
            greedy = A[u - 1] + d
            if greedy < A[v - 1]:
                A[v - 1] = greedy
                Q.update_node_value(v, greedy)
    
    return A

with benchmark("Dijkstra on part 1 problem") as r:
    G = {}
    for line in open("../../../part1/problem_set/6_Dijkstra/dijkstraData.txt", 'r'):
        ls = line.split()
        key = int(ls[0])
        G[key] = []
        for pair in ls[1:]:
            node, distance = pair.split(',')
            p = (int(node), int(distance))
            G[key].append(p)
    
    A = dijkstra(G, 200, 1)

Dijkstra on part 1 problem: 0.0324 seconds


## Recap of Johnson's Algorithm

Given a directed graph $G = (V, E)$ with general edge lengths $c_e$.
1. Form $G'$ by adding a new vertex $s$ and a new edge $(s, v)$ with length 0 for all $v \in G$.

2. Run Bellman-Ford on $G'$ with source vertex $s$.
   Report any negative-cost cycle if detected because it must lie in $G$.
   
3. For each $v \in G$, define $p_v = $ the length of shortest $s \rightarrow v$ path in $G'$.
   Define $c_{u-v}' = c_{u-v} + p_u - p_v$ for each edge $(u, v) \in G$.
   
4. For each vertex $u$ of $G$, compute the shortest path distance $d'[u, v]$ for all $v \in G$ with souce vertex $u$ and edge lengths $\{c_e'\}$ via Dijkstra's algorithm.

5. For each pair $u, v \in G$, return the shortest-path distance $d[u,v] = d'[u,v] - p_u + p_v$.

In [8]:
# This block implements Johnson's algorithm on APSP.
import numbers, collections

def build_extended_graph(nv, G):
    """ Build G' of step 1.
    The new vertex will be the last one, namely nv + 1
    
    Warning: G will be changed.
    """
    
    s = nv + 1
    for v in range(nv):
        G[(s, v + 1)] = 0
    return

def restore_graph(nv, graph):
    """ Restore G from G' for step 3.
    nv -- number of vertices in the original graph G
    graph -- the extended graph G'
    """
    
    s = nv + 1
    for v in range(nv):
        del graph[(s, v + 1)]
    return

def format_graph(graph):
    """ Format graph G from {(tail, head): distance} to {tail: [(head, distance)]}. """
    
    G = collections.defaultdict(list)
    for pair in graph.keys():
        tail, head = pair
        dist = graph.pop(pair)
        G[tail].append((head, dist))
    return G

def johnson(nv, G):
    """ Johnson's algorithm for all-pairs shortest path problem.
    nv -- number of vertices
    G -- graph represented by {(tail, head): distance}
    
    Return the minimum of all-pairs shortest path.
    """
    
    build_extended_graph(nv, G)
    
    p = bellman_ford(s = nv + 1, nv = nv + 1, graph = G) # p is a numpy array of size nv + 1
    if isinstance(p, numbers.Number) and p == float("inf"):
        return float("inf")
    p = p[:-1] # a view of p with s removed
    
    restore_graph(nv, G)
    for tail, head in G.keys():
        G[(tail, head)] += p[tail - 1] - p[head - 1]
    
    G = format_graph(G)
    vmins = []
    for u in range(nv):
        A = dijkstra(G, nv, u + 1) # A is a numpy array of size nv
        P = np.full((nv), p[u], dtype=float)
        P -= p
        A -= P
        A[u] = float("inf")
        vmins.append(np.min(A))
    
    return min(vmins)

In [9]:
# test case -- -242
nv, ne, G = readfile("test.txt")
with benchmark("Johnson on test.txt") as r:
    vmin = johnson(nv, G)
assert vmin == -242, "Johnson does not pass the test case!"
print "Johnson passes the test case!"

Johnson on test.txt: 0.0349 seconds
Johnson passes the test case!


In [10]:
files = ["g1.txt", "g2.txt", "g3.txt"]
files = ["g3.txt"]
for filename in files:
    nv, ne, G = readfile(filename)
    with benchmark("Johnson on {0}".format(filename)) as r:
        vmin = johnson(nv, G)
#         print "Shortest path value of {0} is {1}.".format(filename, vmin)

Johnson on g3.txt: 133 seconds


In [11]:
# Need to find better ways to vectorize Bellman-Ford

# with benchmark("Read file large.txt") as r:
#     nv, ne, G = readfile("large.txt")
# with benchmark("Johnson on large.txt") as r:
#     vmin = johnson(nv, G)
# print "Shortest path value of {0} is {1}.".format("large.txt", vmin)


##########################
# Results on our cluster #
##########################

# Read file large.txt: 2.9 seconds                                                                          
# Johnson on large.txt: 2.61e+04 seconds
# Shortest path value of large.txt is -6.0.